In [1]:
test = {"hello":1}


if "world" not in test:
    print("hello")
else:
    print("Hey")

hello


In [6]:
import torch
import torch.nn as nn



x = torch.randn(size=(10, 1))
y = torch.randint(0, 2, size=(10, 1))

In [8]:
bce_loss = nn.BCEWithLogitsLoss(reduction="none")

In [12]:
loss = bce_loss(x, y.float())


loss.shape

torch.Size([10, 1])

In [17]:
x_ = list(range(10))


list(zip(x, x_))

[(tensor([-0.5817]), 0),
 (tensor([0.6747]), 1),
 (tensor([-1.2691]), 2),
 (tensor([0.9205]), 3),
 (tensor([-0.6124]), 4),
 (tensor([-0.0104]), 5),
 (tensor([-0.7992]), 6),
 (tensor([0.3683]), 7),
 (tensor([1.3672]), 8),
 (tensor([-0.0916]), 9)]

In [18]:
from src.model import *


conv = get_convnext_tiny()

In [20]:
sum(torch.numel(param) for param in conv.parameters())

27830894

In [21]:
vit = get_vit_tiny()

In [23]:
type(vit)

timm.models.vision_transformer.VisionTransformer